In [1]:
import pandas as pd
import sklearn as ktl

In [2]:
bol = pd.read_csv("/Users/hendrixperalta/Desktop/bolivia/data/GeoDS4Bolivia.csv")
#list(bol.columns)

#'imds'
#"ln_NTLpc2012"


In [3]:
x = bol[["ln_NTLpc2012", "ln_t400NTLpc2012", "pop2012", ]].fillna(0)
y = bol['imds']

In [4]:
x.dtypes

ln_NTLpc2012        float64
ln_t400NTLpc2012    float64
pop2012             float64
dtype: object

In [5]:
y.dtypes

dtype('float64')

In [6]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()

clf.get_params()



{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [7]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3) 

In [8]:
#clf.fit(x_train,y_train)

In [9]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [10]:
model.fit(x_train,y_train)

LinearRegression()

In [11]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R^2 Score:", r2)

Mean Squared Error: 36.27054270773723
R^2 Score: 0.05290471842945321
